# Nieuwe API's genereren

Met dit notebook kun je op basis van delen van het [Gemeentelijk Gegevensmodel](https://github.com/Gemeente-Delft/Gemeentelijk-Gegevensmodel) een API-implementatie genereren. Hiervoor worden drie bestanden gegenereerd binnen de de context van een Docker Flask App. Het gaat om de volgende besatanden:

1. models.py
2. filters.py
3. schema.py

Even opnieuw opstarten en je API is live!

Om te kiezen welk onderdeel van het GGM je wil uitgeneren kies je het GUID van het bijbehorende package. Alles wat daaronder zit genereert deze app uit. Pas op dat je niet teveel doet. Als voorbeeld zijn de GUID's van Monumenten en Onderwijs toegevoegd. Deze sheet werkt op basis van het GGM in een Excel-bestand, hij kan ook direct de onderliggende database van Enterprise Architect benaderen maar dat werkt alleen onder Windows en sgtaat standaard uit.   

In [1]:
### Importeer bibliotheken en utils

import os
import pandas as pd
import json
from IPython.display import JSON as JSONDisplay
import requests
import database
from re import sub
import ast

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

#Utility
def snake_case(s):
  return '_'.join(
    sub('([A-Z][a-z]+)', r' \1',
    sub('([A-Z]+)', r' \1',
    s.replace('-', ' '))).split()).lower()
def convert(s):
    s = str(s)
    return snake_case(s)

## Configuratie

Met de waarden hieronder configureer je de generatie. Als voorbeeld is het GUID van Onderwijs actief. 

In [2]:
### Configuratie
db_uri = os.getenv("DATABASE_GGM_URL")

root_guid = '{88E4851C-DACE-4cec-9B14-A36FA6D3622E}' #Onderwijs in package ontwikkeling
#root_guid = '{5BF1D9DB-1013-421d-86E1-63A30028333E}' #Monumenten

models_file = "/opt/project/models.py"
filters_file = "/opt/project/filters.py"
schema_file = "/opt/project/schema.py"

## Inlezen GGM

Hieronder wordt het GGM ingelezen in het dataframe df voor verdere verwerking. Als het goed is toont deze cel de eerste 5 regels uit het GGM.

In [3]:
df = database.get_df_complete(db_uri, root_guid=root_guid)

# find all classnames
lst_classnames = df[df.object_type == 'Class']['name'].unique()
lst_classnames = [convert(item) for item in lst_classnames if item]

# find all enums
lst_enum = df[df.object_type == 'Enumeration']['name'].unique()
lst_enum = [convert(item) for item in lst_enum]

# find all n:m relations
lst_nm = []
for sublst in list(df['start_connectors']):
    lst_nm = lst_nm + sublst

lst_nm= [elem for elem in lst_nm if 
         not elem['connector_sourcecard'] is None 
         and '*' in elem['connector_sourcecard'] 
         and not elem['connector_destcard'] is None 
         and '*' in elem['connector_destcard']
         and convert(elem['end_object_name']) in lst_classnames]
df.head(5)

,object_type,name,alias,author,version,objectnote,ea_guid,modifieddate,attributes,start_connectors,end_connectors,tree
972,Class,School,NaN,crossover,1.0,NaN,{8DA53147-9C22-4a15-9E2E-F820BDC09AE3},2021-09-15 11:26:45,"[{'attribute_name': 'naam', 'attribute_type': ...","[{'connector_id': 1371.0, 'connector_name': 'g...","[{'connector_id': 1386.0, 'connector_name': 'h...",446-445-438-165
973,Class,Startkwalificatie,NaN,crossover,1.0,NaN,{47B33A86-9F0E-4f48-A017-F0D29C70F6A0},2021-09-15 11:26:45,"[{'attribute_name': 'datumbehaald', 'attribute...",[],"[{'connector_id': 1383.0, 'connector_name': 'h...",446-445-438-165
974,Class,Uitschrijving,NaN,crossover,1.0,NaN,{F51FC3E7-6084-4e2b-92B6-23118BB32666},2021-09-15 11:26:45,"[{'attribute_name': 'datum', 'attribute_type':...",[],"[{'connector_id': 1372.0, 'connector_name': 'h...",446-445-438-165
975,Enumeration,Onderwijstype,NaN,crossover,1.0,NaN,{1B214976-0BFE-47a2-B229-98B797A0152F},2021-09-15 11:26:45,"[{'attribute_name': 'VMBO-T', 'attribute_type'...",[],[],446-445-438-165
976,Class,Inschrijving,NaN,crossover,1.0,NaN,{6A6B83EA-33D0-4b30-862C-AB7328E2B795},2021-09-15 11:26:44,"[{'attribute_name': 'datum', 'attribute_type':...","[{'connector_id': 1370.0, 'connector_name': No...","[{'connector_id': 1385.0, 'connector_name': 'h...",446-445-438-165


## Genereer models.py

In models.py worden de datadefinities uitgegenereerd conform [SQLAlchemy](https://www.sqlalchemy.org). Er wordt gebruik gemaakt van relatief eenvoudige templates op basis van de [f-string](https://realpython.com/python-f-strings/) binnen Python. 

In [4]:

#### generate enums
def getEnumValues(lst):    
    output = ""
    for attr in lst:
        attributename = attr['attribute_name']
        if attributename is None:
            return ""
        attributename = convert(attributename).title()
        template_attribute=f'    {attributename} = "{attributename}"\n'
        output = output + template_attribute
    return output
    
def getEnumerations(df):
    output = ""
    for i, clas in df.iterrows():
        classname = convert(clas['name'])
        
        template_class =f'''
class {classname}Enum(enum.Enum):
{getEnumValues(clas['attributes'])}
        '''        
        output = output + template_class
    return output

##### Associaltin table for n:m relations
def getReltables():
    output = ""
    for rel in lst_nm:
        left_name = convert(rel['start_object_name'])
        right_name = convert(rel['end_object_name'])
        name = left_name + "_" + right_name + "_table"
        output = output + f'''
{name} = Table("{name}", Base.metadata,
    Column("{left_name}_id", ForeignKey("{left_name}.id", deferrable=True), primary_key=True),
    Column("{right_name}_id", ForeignKey("{right_name}.id", deferrable=True), primary_key=True)
)'''        
    return output



###### Classes, attributes and relations
def getRelations(lst_start, lst_end): 
    output = ""
    for attr in lst_start: 
        if attr['connector_destcard'] in ['0','1','0..1'] and not attr['connector_id'] is None and attr['connector_type'] == 'Association':
            relationname = attr['end_object_name']
            relationname = convert(relationname)
            output = output + f'    {relationname}ID = Column(ForeignKey("{relationname}.id", deferrable=True), index=True{", nullable=False" if attr["connector_destcard"] == "1" else ""})\n'
            output = output + f'    {relationname} = relationship("{relationname}", backref="{convert(attr["start_object_name"])}")\n'
        if attr in lst_nm:
            left_name = convert(attr['start_object_name'])
            right_name = convert(attr['end_object_name'])
            tablename = left_name + "_" + right_name + "_table"            
            output = output + f'    {right_name} = relationship("{right_name}", secondary={tablename}, backref="{left_name}")\n'
    for attr in lst_end: 
        if attr['connector_sourcecard'] in ['0','1','0..1'] and not attr['connector_id'] is None and attr['connector_type'] == 'Association' and not attr['connector_destcard'] in ['0','1','0..1']:
            relationname = attr['start_object_name']
            relationname = convert(relationname)
            output = output + f'    {relationname}ID = Column(ForeignKey("{relationname}.id", deferrable=True), index=True{", nullable=False" if attr["connector_sourcecard"] == "1" else ""})\n'
            output = output + f'    {relationname} = relationship("{relationname}", backref="{convert(attr["end_object_name"])}")\n'
    return output

#Removed #"Enum("+ convert(attr["attribute_type"]) +"Enum)" if convert(attr["attribute_type"]) in lst_enum else \
def getattributes(lst):
    output = ""
    for attr in lst:
        attributename = attr['attribute_name']
        if attributename is None:
            return ""
        attributename = convert(attributename)
        column_type = "Date" if attr["attribute_type"] == "Datum" else \
                      "Integer" if attr["attribute_type"] == "int" else \
                      "Boolean" if attr["attribute_type"] == "boolean" else \
                      "String"
        template_attribute=f'    {attributename} = Column({column_type})\n'
        output = output + template_attribute
    return output
    
def getClasses(df):
    output = ""
    for i, clas in df.iterrows():
        classname = convert(clas['name'])
        
        template_class =f'''
class {classname}(Base):
    __tablename__ = "{classname.lower()}"
    id = Column(Integer, primary_key=True)  
{getattributes(clas['attributes'])}
{getRelations(clas['start_connectors'], clas['end_connectors'])}
        '''
        
        output = output + template_class
    return output



    
template_model = f'''
from project.database import Base
from sqlalchemy import Column, DateTime, ForeignKey, Integer, String, func, Boolean, Date, Enum, Table
from sqlalchemy.orm import backref, relationship
import enum


{getEnumerations(df[df.object_type == 'Enumeration'])}

{getReltables()}

{getClasses(df[df.object_type == 'Class'])}
'''

generated_model = template_model

f = open(models_file, "w")
f.write(generated_model)
f.close()

## Genereer filters.py

In filters.py staan alle filters die binnen de GraphQL-queries kunnen worden gebruikt. 

In [5]:

def getAttributeFilters(lst):
    lst = lst + [{"attribute_name": "id"}]
    lst = [f'            "{convert(attr["attribute_name"])}": [...]' for attr in lst if not attr["attribute_name"] is None]
    return ",\n".join(lst)                              

def getClassFilters(df): 
    output = ""
    for i, clas in df.iterrows():
        classname = convert(clas['name'])
        
        template_class =f'''
class {classname}Filter(FilterSet):
    class Meta:
        model = {classname}
        fields = {{
{getAttributeFilters(clas['attributes'])} }}
'''
        output = output + template_class
    return output



template_filters = f'''
from graphene_sqlalchemy_filter import FilterableConnectionField, FilterSet
from project.models import {", ".join(lst_classnames)}

{getClassFilters(df[df.object_type == 'Class'])}

class MyFilterableConnectionField(FilterableConnectionField):
    filters = {{{", ".join([cl + ": " + cl+"Filter()" for cl in lst_classnames])}}}

'''

template_filters

f = open(filters_file, "w")
f.write(template_filters)
f.close()

## Genereer schema.py

In schema.py staan de te publiceren GraphQL-services. 

In [6]:
work_lst = lst_classnames


def getClasses(lst):
    output = ""
    for cl in lst: 
        
        output = output + f'''    
class {cl}(SQLAlchemyObjectType):
    class Meta:
        model = {cl}Model
        interfaces = (relay.Node, )
        connection_field_factory = MyFilterableConnectionField.factory
        '''
    return output
        

str_import = "\n".join(["from project.models import "+ elem +" as "+ elem +"Model" for elem in work_lst]) 
str_querues = "\n".join([f'    {elem} = MyFilterableConnectionField({elem}.connection, sort={elem}.sort_argument())' for elem in work_lst])

template_filters = f'''
import graphene
from graphene import relay
from graphene_sqlalchemy import SQLAlchemyConnectionField, SQLAlchemyObjectType

{str_import}
from project.filters import MyFilterableConnectionField

{getClasses(work_lst)}

class Query(graphene.ObjectType):
    node = relay.Node.Field()
    # Allow only single column sorting
{str_querues}


schema = graphene.Schema(query=Query)

'''

template_filters

f = open(schema_file, "w")
f.write(template_filters)
f.close()